Import the libraries

In [94]:
from dotenv import load_dotenv
from langchain import hub
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import os
import datetime
import urllib

load_dotenv()

True

Define tools

In [104]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from test.test_typing import Annotated
import smtplib, ssl

@tool("get_current_datetime")
def get_current_datetime() -> str :
    """Get current date and time"""
    return datetime.datetime.now().strftime('%m/%d/%Y, %H:%M:%S')

@tool("convert_to_japanse_datetime_custom")
def convert_to_japanse_datetime(date_str: Annotated[str, 'date string in ISO format']) -> str :
    """Convert date string to custom japanese date time"""
    dt = datetime.datetime.fromisoformat(date_str)
    return f"{dt.year}年{dt.month}月{dt.day}日　{dt.hour}時{dt.minute}分{dt.second}秒"

# dir: Annotated[str, 'the directory of the file location, pass in /tmp for default']
@tool("download_image_from_url")
def download_image_from_url(url: Annotated[str, 'full URL of the web image']) -> dict:
    "Download an image from web and save to local device"
    directory = f'{os.getcwd()}/tmp'
    file_name = f"{directory}/{datetime.datetime.timestamp(datetime.datetime.now())}-agent.png"
    cleaned_url = url.replace("'", '')
    try:
        res= urllib.request.urlretrieve(cleaned_url,file_name)    
        print(res)
        os.startfile(file_name)
        return  {
            "success": True,
            "file_name": file_name
        }
    except (Exception) as e:
         return {
                "success": False,
                "error": f"{e}",
                "url": url
            }
         
@tool("send_email")
def send_email(to: Annotated[str, 'the receiver\'s email address'], name: Annotated[str, 'the receiver\'s name'], content: Annotated[str, 'the content of the email'], attachment_paths: Annotated[list[str], 'full paths of the files to be attached to the mail, default to an empty list']) -> dict :
    """Send an email to specified email address, with name, content and optional attachments"""
    msg = MIMEMultipart()
    msg['Subject'] = 'Generated By AI Agent'
    msg['From'] = os.getenv('SMTP_USERNAME')
    msg['To'] = to
    msg.attach(MIMEText(f'[Powered by Benihayashi\'s AI Agent] \n{content}', "plain"))
    print(msg)
    try:
        server = smtplib.SMTP(os.getenv('SMTP_HOST'),587)
        server.starttls() 
        server.login(os.getenv('SMTP_USERNAME'), os.getenv('SMTP_PASSWORD'))
        print(msg['From'], msg['To'], msg.as_string())
        server.sendmail(msg['From'], msg['To'], msg.as_string())
        server.quit()
        return {
            "success": True,
            "subject": msg['Subject'],
        }
    except Exception as e:
        print(e)
        return {
            "success": False,
            "error": e,
        }

    
    

Initialized OpenAI LLM

In [105]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv('OPENAI_KEY'),
)

tools = [
    get_current_datetime,
    convert_to_japanse_datetime,
    download_image_from_url,
    send_email,
]

# prompt = hub.pull("hwchase17/react",api_key=os.getenv('LANGSMITH_KEY'),)
memory = MemorySaver()
agent = create_react_agent(
    llm,
    tools=tools,
    # prompt=prompt,
    checkpointer=memory,
)


Testing

In [106]:
import uuid
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": uuid.uuid4()}}
#prompt1 = "Download this image for me: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s"
prompt1= "Send an email to bennylim0403@gmail.com, his name is Benny Lim, give him a simple greetings"
response = agent.invoke({"messages": [HumanMessage(prompt1)]},config=config)
print("question: ", prompt1)
print("response: ", response['messages'][-1].content)
# for event in agent_executor.stream({"input": [HumanMessage(content=prompt1)]}, config):
#     print("question: ", prompt1)
#     print("response: ", event['output'])

Content-Type: multipart/mixed; boundary="===============5455770373450155234=="
MIME-Version: 1.0
Subject: Generated By AI Agent
From: benny.lim@tresoftsys.com
To: bennylim0403@gmail.com

--===============5455770373450155234==
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

[Powered by Benihayashi's AI Agent] 
Hello Benny! Hope you are doing well.
--===============5455770373450155234==--

benny.lim@tresoftsys.com bennylim0403@gmail.com Content-Type: multipart/mixed; boundary="===============5455770373450155234=="
MIME-Version: 1.0
Subject: Generated By AI Agent
From: benny.lim@tresoftsys.com
To: bennylim0403@gmail.com

--===============5455770373450155234==
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

[Powered by Benihayashi's AI Agent] 
Hello Benny! Hope you are doing well.
--===============5455770373450155234==--

question:  Send an email to bennylim0403@gmail.com, his name is Benny Lim,

In [62]:
prompt2 = "can you download the file again?"
response = agent.invoke({"messages": [HumanMessage(prompt2)]},config=config)
print("question: ", prompt2)
print("response: ", response['messages'][-1].content)

('c:\\Users\\USER\\my-projects\\movie_react_agent/tmp/1740579163.658552-agent.png', <http.client.HTTPMessage object at 0x0000023B9F3600A0>)
question:  can you download the file again?
response:  The image has been successfully downloaded again and saved as "1740579163.658552-agent.png" in the specified directory.
